In [1]:
!pip install transformers simpletransformers nltk


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install datasets



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install --upgrade --force-reinstall datasets

  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.1.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pyarrow-18.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.11.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached huggingface_hub-0.26.3-py3

In [4]:
pip install datasets

DEPRECATION: Loading egg at /home/akhil/anaconda3/lib/python3.11/site-packages/huggingface_hub-0.26.2-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [5]:
# import the necessary libraries
import os
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

/home/akhil/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# read the dataset
df = pd.read_csv("dataset.csv")
df

,Text,labels
0,"Yes, this product sounds great. I would like t...",positive
1,I've been looking for something like this. Tel...,positive
2,This seems useful. Please provide more details.,positive
3,"Sure, I’m interested in your product.",positive
4,I’m happy with the offer. How do I proceed?,positive
...,...,...
295,I’ll need to review this later and get back to...,neutral
296,I’ll review everything you’ve shared with me b...,neutral
297,"I appreciate the call, but I’m not ready to ma...",neutral
298,I’ll need to look into your services more befo...,neutral


In [7]:
# print the column names
df.columns

Index(['Text', 'labels'], dtype='object')

In [8]:
# get the value count for each class/labels
df['labels'].value_counts()

labels
positive    100
negative    100
neutral     100
Name: count, dtype: int64

In [9]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['labels'])


In [10]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (240, 2), eval is (60, 2)


In [11]:
train['labels'].value_counts()

labels
negative    80
neutral     80
positive    80
Name: count, dtype: int64

In [12]:
eval['labels'].value_counts()

labels
negative    20
positive    20
neutral     20
Name: count, dtype: int64

In [13]:
# Save splitted subsets

train.to_csv("train_subset.csv", index=False)
eval.to_csv("eval_subset.csv", index=False)

In [15]:
dataset = load_dataset('csv',
                        data_files={'train': 'train_subset.csv',
                        'eval': 'eval_subset.csv'}, encoding = "ISO-8859-1")

Generating train split: 240 examples [00:00, 28634.13 examples/s]
Generating eval split: 60 examples [00:00, 13998.12 examples/s]


In [16]:
#create a function to convert label
def transform_labels(label):

    label = label['labels']
    num = 0
    if label == 'negative':
        num = 0
    elif label == 'neutral':
        num = 1
    elif label == 'positive':
        num = 2

    return {'labels': num}


# Transform labels and remove the useless columns
dataset = dataset.map(transform_labels)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 6561.12 examples/s]


In [17]:
from dotenv import load_dotenv
import os

In [18]:
# Load environment variables from .env
load_dotenv()

# Get the Hugging Face token
hf_auth_token = os.getenv("hf_token")

In [20]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncase')

tokenizer = AutoTokenizer.from_pretrained(
    'distilbert-base-uncased',
    use_auth_token = hf_auth_token
)


# let's tokenize the data for the model to be able to understand
def tokenize_data(example):
    return tokenizer(example['Text'], padding='max_length')

/home/akhil/anaconda3/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [21]:
dataset = dataset.map(tokenize_data, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 2380.99 examples/s]


In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 240
    })
    eval: Dataset({
        features: ['Text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 60
    })
})

In [23]:
from transformers import AutoModelForSequenceClassification

# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                            num_labels=3,
                                                            use_auth_token = hf_auth_token)

/home/akhil/anaconda3/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-12-03 09:27:40.506151: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 09:27:40.556611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 09:27:40.556656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 09:27:40.557680: E external/lo

In [24]:
# let's set the training arguements

# the default batch size for training arguments
batch_size = 8

# set number of epochs
number_of_epochs = 7
# let set the logging steps
logging_steps = len(dataset['train']) // batch_size # it should log each batch 

steps = (len(dataset['train']) / batch_size) * number_of_epochs
warmup_steps = int(0.2 * steps)



from transformers import TrainingArguments
training_args = TrainingArguments(
                                  num_train_epochs=number_of_epochs, 
                                  load_best_model_at_end=True,
                                  evaluation_strategy='steps', 
                                  save_strategy='steps',
                                  learning_rate=2e-5,
                                  logging_steps=logging_steps,
                                  warmup_steps= warmup_steps,
                                  save_steps=1000,
                                  eval_steps=500,
                                  output_dir="fine-tuned-distilbert-base-uncased"
                                  )

/home/akhil/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
# shuffle the datasets

train_dataset = dataset['train'].shuffle(seed=10) 
eval_dataset = dataset['eval'].shuffle(seed=10)

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [27]:
# Launch the learning process: training 
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=210, training_loss=0.38111157672745843, metrics={'train_runtime': 1679.9491, 'train_samples_per_second': 1.0, 'train_steps_per_second': 0.125, 'total_flos': 222549198520320.0, 'train_loss': 0.38111157672745843, 'epoch': 7.0})

In [28]:
# save the model locally
trainer.save_model("./models/")

In [29]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [30]:
# save the tokeniser locally
tokenizer.save_pretrained("distilbert-base-uncased")

('distilbert-base-uncased/tokenizer_config.json',
 'distilbert-base-uncased/special_tokens_map.json',
 'distilbert-base-uncased/vocab.txt',
 'distilbert-base-uncased/added_tokens.json',
 'distilbert-base-uncased/tokenizer.json')

In [31]:
pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /home/akhil/anaconda3/lib/python3.11/site-packages/huggingface_hub-0.26.2-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [32]:
import numpy as np
from sklearn.metrics import mean_squared_error
import evaluate

def compute_metrics(eval_pred):
    # Load the metrics to use
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate the metrics using the predicted and true values
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {"accuracy": accuracy["accuracy"], "f1score": f1["f1"]}


In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

In [34]:
# Evaluate the fine-tuned model 
trainer.evaluate()

{'eval_loss': 0.22252653539180756,
 'eval_model_preparation_time': 0.0026,
 'eval_accuracy': 0.9333333333333333,
 'eval_f1score': 0.9328955597248282,
 'eval_runtime': 23.4251,
 'eval_samples_per_second': 2.561,
 'eval_steps_per_second': 0.342}

In [35]:
!pip install huggingface_hub


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [36]:
from huggingface_hub import login

# Log in using your Hugging Face API token
login(token=hf_auth_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/akhil/.cache/huggingface/token
Login successful


In [37]:
from huggingface_hub import whoami

# Check if you're logged in
user_info = whoami()
print(user_info)


{'type': 'user', 'id': '661e4fff80a4867a5518b9ce', 'name': 'AkhilJX', 'fullname': 'K Akhil JX', 'isPro': False, 'avatarUrl': '/avatars/7b70827e6cdafde1dd6716004c5b9706.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'sentiment_analysis_test', 'role': 'fineGrained', 'createdAt': '2024-11-28T05:47:43.062Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '661e4fff80a4867a5518b9ce', 'type': 'user', 'name': 'AkhilJX'}, 'permissions': ['repo.content.read', 'repo.write']}]}}}}


In [38]:
# push your model and tokenizer to the hugging face. 
trainer.push_to_hub()

events.out.tfevents.1733198272.LT-090.29620.0:   0%|                                                                                                                                   | 0.00/6.94k [00:00<?, ?B/s]
model.safetensors:   0%|                                                                                                                                                                | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|                                                                                                                                                               | 0.00/5.30k [00:00<?, ?B/s]


events.out.tfevents.1733199979.LT-090.29620.1:   0%|                                                                                                                                     | 0.00/457 [00:00<?, ?B/s]



Upload 4 LFS files:   0%|                                                                                                                         

CommitInfo(commit_url='https://huggingface.co/AkhilJX/fine-tuned-distilbert-base-uncased/commit/8471f4a950bbd64fd722fe04387005e7d22553af', commit_message='End of training', commit_description='', oid='8471f4a950bbd64fd722fe04387005e7d22553af', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AkhilJX/fine-tuned-distilbert-base-uncased', endpoint='https://huggingface.co', repo_type='model', repo_id='AkhilJX/fine-tuned-distilbert-base-uncased'), pr_revision=None, pr_num=None)

In [39]:
# curl https://huggingface.co/AkhilJX/fine-tuned-distilbert-base-uncased

# verify if the model was uploaded properly
import requests

# Define the URL
url = "https://huggingface.co/AkhilJX/fine-tuned-distilbert-base-uncased"

# Make the GET request
response = requests.get(url)

# Check the response status code
if response.status_code == 200:
    print("Request was successful.")
    # Print a portion of the response text
    print(response.text[:500])  # Display the first 500 characters of the response
else:
    print(f"Request failed with status code: {response.status_code}")


Request was successful.
<!doctype html>
<html class="">
	<head>
		<meta charset="utf-8" />
		<meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=no" />
		<meta name="description" content="We’re on a journey to advance and democratize artificial intelligence through open source and open science." />
		<meta property="fb:app_id" content="1321688464574422" />
		<meta name="twitter:card" content="summary_large_image" />
		<meta name="twitter:site" content="@huggingface" />
		<meta name="twit


In [40]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("AkhilJX/fine-tuned-distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("AkhilJX/fine-tuned-distilbert-base-uncased")

In [41]:
# test the model on sample text
import torch

# Function for sentiment analysis
def analyze_sentiment(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Get model predictions
    outputs = model(**inputs)
    logits = outputs.logits
    # Get predicted class (highest logit score)
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class

# Example usage
text = "I’m just browsing for now, not looking to make any decisions yet."
sentiment = analyze_sentiment(text)

# Interpret sentiment (classification: 0 = negative, 1 = neutral, 2 = positive)
if sentiment == 0:
    print("Sentiment: Negative")
elif sentiment == 1:
    print("Sentiment: Neutral")
elif sentiment == 2:
    print("Sentiment: Positive")
else:
    print(f"Sentiment: Class {sentiment}")

Sentiment: Neutral


# ........................................................Thank You.....................................................